## Imports

In [1]:
from datetime import datetime
import numpy as np
import pickle
import json
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!cd gdrive/MyDrive/SPARSe

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load project and donation data

In [3]:
donations = pd.read_csv("./gdrive/MyDrive/SPARSe/data/donations.csv")
projects = pd.read_csv("./gdrive/MyDrive/SPARSe/data/projects.csv")
# essays = pd.read_csv("./gdrive/MyDrive/SPARSe/data/essays.csv")

In [4]:
#projects_donations_df - merge between projects and donations
projects_donations_df = projects.merge(donations, on='projectid', how='left')
projects_donations_df['date_posted'] = projects_donations_df.date_posted.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
projects_donations_df['deadline'] = projects_donations_df['date_posted'] + pd.DateOffset(months=4)

In [5]:
# projects_donations_filtered_df - Filter to donations before deadline
projects_donations_filtered_df = projects_donations_df[projects_donations_df.donation_timestamp <= projects_donations_df.deadline]

# find evaluation date
projects_donations_filtered_df['date_posted_plus_4_weeks'] = projects_donations_filtered_df['date_posted'] + pd.DateOffset(weeks=4)
def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d +  pd.DateOffset(days=days_ahead)

projects_donations_filtered_df['evaluation_date'] = projects_donations_filtered_df.date_posted_plus_4_weeks.apply(lambda x: next_weekday(x, 0))

<ipython-input-5-5e91f05d93d7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects_donations_filtered_df['date_posted_plus_4_weeks'] = projects_donations_filtered_df['date_posted'] + pd.DateOffset(weeks=4)
<ipython-input-5-5e91f05d93d7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects_donations_filtered_df['evaluation_date'] = projects_donations_filtered_df.date_posted_plus_4_weeks.apply(lambda x: next_weekday(x, 0))


In [29]:
project_features_df.head()

,projectid,school_state,school_metro,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_area,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_excluding_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match
0,316ed8fb3b81402ff6ac8f721bb31192,CA,NaN,f,f,f,f,f,f,Mrs.,f,f,Literacy & Language,Applied Learning,Books,highest poverty,Grades 6-8,555.81,32.0,f,f
1,90de744e368a7e4883223ca49318ae30,TX,urban,f,f,f,f,f,f,Mrs.,f,f,Literacy & Language,Literacy & Language,Books,highest poverty,Grades PreK-2,296.47,22.0,f,f
2,32943bb1063267de6ed19fc0ceb4b9a7,OR,rural,f,f,f,f,f,f,Mr.,f,f,Literacy & Language,Math & Science,Technology,high poverty,Grades PreK-2,430.89,17.0,f,f
3,bb18f409abda2f264d5acda8cab577a9,NY,urban,f,t,f,f,f,f,Mr.,t,f,History & Civics,Special Needs,Books,highest poverty,Grades 3-5,576.07,12.0,f,f
4,24761b686e18e5eace634607acbcc19f,CA,urban,f,f,f,f,f,f,Ms.,f,f,Math & Science,Literacy & Language,Other,highest poverty,Grades PreK-2,408.40,24.0,f,f


In [63]:
projects_donations_filtered_df['date_posted_plus_1_weeks'] = projects_donations_filtered_df['date_posted'] + pd.DateOffset(weeks=1)
projects_donations_filtered_df['date_posted_plus_2_weeks'] = projects_donations_filtered_df['date_posted'] + pd.DateOffset(weeks=2)
projects_donations_filtered_df['date_posted_plus_3_weeks'] = projects_donations_filtered_df['date_posted'] + pd.DateOffset(weeks=3)
projects_donations_filtered_df['date_5w_num_days'] = (projects_donations_filtered_df['evaluation_date'] - projects_donations_filtered_df['date_posted_plus_4_weeks']).apply(lambda x: x.days)

<ipython-input-63-f3d39d8f0094>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects_donations_filtered_df['date_posted_plus_1_weeks'] = projects_donations_filtered_df['date_posted'] + pd.DateOffset(weeks=1)
<ipython-input-63-f3d39d8f0094>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects_donations_filtered_df['date_posted_plus_2_weeks'] = projects_donations_filtered_df['date_posted'] + pd.DateOffset(weeks=2)
<ipython-input-63-f3d39d8f0094>:3: SettingWithCopyWarning: 
A value is trying to be se

In [87]:
trend_df = projects[['projectid', 'date_posted']]
trend_df['date_posted'] = trend_df.date_posted.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
trend_df['date_posted_plus_1_weeks'] = trend_df['date_posted'] + pd.DateOffset(weeks=1)
trend_df['date_posted_plus_2_weeks'] = trend_df['date_posted'] + pd.DateOffset(weeks=2)
trend_df['date_posted_plus_3_weeks'] = trend_df['date_posted'] + pd.DateOffset(weeks=3)
trend_df['date_posted_plus_4_weeks'] = trend_df['date_posted'] + pd.DateOffset(weeks=4)
trend_df['evaluation_date'] = trend_df.date_posted_plus_4_weeks.apply(lambda x: next_weekday(x, 0))
trend_df['date_5w_num_days'] = (trend_df['evaluation_date'] - trend_df['date_posted_plus_4_weeks']).apply(lambda x: x.days)
trend_df.head()

<ipython-input-87-ad3190ee3c12>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trend_df['date_posted'] = trend_df.date_posted.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
<ipython-input-87-ad3190ee3c12>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trend_df['date_posted_plus_1_weeks'] = trend_df['date_posted'] + pd.DateOffset(weeks=1)
<ipython-input-87-ad3190ee3c12>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1


In [88]:
trend_df.head()

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1


In [89]:
# get 1W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_1_weeks].groupby('projectid').donation_to_project.sum().reset_index(name='1W_donations'), on='projectid', how='left').fillna(0)

# get 2W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_2_weeks].groupby('projectid').donation_to_project.sum().reset_index(name='2W_donations'), on='projectid', how='left').fillna(0)

# get 3W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_3_weeks].groupby('projectid').donation_to_project.sum().reset_index(name='3W_donations'), on='projectid', how='left').fillna(0)

# get 4W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_4_weeks].groupby('projectid').donation_to_project.sum().reset_index(name='4W_donations'), on='projectid', how='left').fillna(0)

# get 5W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.evaluation_date].groupby('projectid').donation_to_project.sum().reset_index(name='5W_donations'), on='projectid', how='left').fillna(0)

trend_df['5W_donations'] -= trend_df['4W_donations']
trend_df['4W_donations'] -= trend_df['3W_donations']
trend_df['3W_donations'] -= trend_df['2W_donations']
trend_df['2W_donations'] -= trend_df['1W_donations']

trend_df

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
664093,a7236ea96c812895cafc5d700d779147,2002-09-17,2002-09-24,2002-10-01,2002-10-08,2002-10-15,2002-10-21,6,254.0,0.0,0.0,0.0,0.0
664094,e02da37beb332eb66c2d2ba989c597ad,2002-09-17,2002-09-24,2002-10-01,2002-10-08,2002-10-15,2002-10-21,6,1241.0,0.0,0.0,0.0,0.0
664095,82e536f14eadf2671a70e03416f695a3,2002-09-16,2002-09-23,2002-09-30,2002-10-07,2002-10-14,2002-10-21,7,125.0,0.0,0.0,0.0,0.0
664096,e139df754a873a62d93daa56acbf8040,2002-09-13,2002-09-20,2002-09-27,2002-10-04,2002-10-11,2002-10-14,3,125.0,0.0,0.0,0.0,0.0


In [90]:
trend_df['5W_donations_per'] = trend_df['5W_donations']/trend_df['date_5w_num_days']
trend_df['4W_donations_per'] = trend_df['4W_donations']/7
trend_df['3W_donations_per'] = trend_df['3W_donations']/7
trend_df['2W_donations_per'] = trend_df['2W_donations']/7
trend_df['1W_donations_per'] = trend_df['1W_donations']/7

trend_df.head()

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations,5W_donations_per,4W_donations_per,3W_donations_per,2W_donations_per,1W_donations_per
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
trend_df['mean_donations_per'] = trend_df[['5W_donations_per', '4W_donations_per', '3W_donations_per', '2W_donations_per', '1W_donations_per']].mean(axis=1)

trend_df.head()

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations,5W_donations_per,4W_donations_per,3W_donations_per,2W_donations_per,1W_donations_per,mean_donations_per
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
trend_df['std_donations_per'] = trend_df[['5W_donations_per', '4W_donations_per', '3W_donations_per', '2W_donations_per', '1W_donations_per']].std(axis=1)
trend_df.head()

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations,5W_donations_per,4W_donations_per,3W_donations_per,2W_donations_per,1W_donations_per,mean_donations_per,std_donations_per
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
trend_df['1W_normalised'] = ((trend_df['1W_donations_per'] - trend_df['mean_donations_per'])/trend_df['std_donations_per']).fillna(0)
trend_df['2W_normalised'] = ((trend_df['2W_donations_per'] - trend_df['mean_donations_per'])/trend_df['std_donations_per']).fillna(0)
trend_df['3W_normalised'] = ((trend_df['3W_donations_per'] - trend_df['mean_donations_per'])/trend_df['std_donations_per']).fillna(0)
trend_df['4W_normalised'] = ((trend_df['4W_donations_per'] - trend_df['mean_donations_per'])/trend_df['std_donations_per']).fillna(0)
trend_df['5W_normalised'] = ((trend_df['5W_donations_per'] - trend_df['mean_donations_per'])/trend_df['std_donations_per']).fillna(0)
trend_df.head()

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations,5W_donations_per,4W_donations_per,3W_donations_per,2W_donations_per,1W_donations_per,mean_donations_per,std_donations_per,1W_normalised,2W_normalised,3W_normalised,4W_normalised,5W_normalised
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
trend_df['trend'] = (-2 * trend_df['1W_normalised'] + -1 * trend_df['2W_normalised'] + 1 * trend_df['4W_normalised'] + 2 * trend_df['5W_normalised'])/10
trend_df

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations,5W_donations_per,4W_donations_per,3W_donations_per,2W_donations_per,1W_donations_per,mean_donations_per,std_donations_per,1W_normalised,2W_normalised,3W_normalised,4W_normalised,5W_normalised,trend
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664093,a7236ea96c812895cafc5d700d779147,2002-09-17,2002-09-24,2002-10-01,2002-10-08,2002-10-15,2002-10-21,6,254.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.285714,7.257143,16.227465,1.788854,-0.447214,-0.447214,-0.447214,-0.447214,-0.447214
664094,e02da37beb332eb66c2d2ba989c597ad,2002-09-17,2002-09-24,2002-10-01,2002-10-08,2002-10-15,2002-10-21,6,1241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177.285714,35.457143,79.284582,1.788854,-0.447214,-0.447214,-0.447214,-0.447214,-0.447214
664095,82e536f14eadf2671a70e03416f695a3,2002-09-16,2002-09-23,2002-09-30,2002-10-07,2002-10-14,2002-10-21,7,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.857143,3.571429,7.985957,1.788854,-0.447214,-0.447214,-0.447214,-0.447214,-0.447214
664096,e139df754a873a62d93daa56acbf8040,2002-09-13,2002-09-20,2002-09-27,2002-10-04,2002-10-11,2002-10-14,3,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.857143,3.571429,7.985957,1.788854,-0.447214,-0.447214,-0.447214,-0.447214,-0.447214


In [102]:
trend_df[trend_df['trend'] > 0]

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations,5W_donations_per,4W_donations_per,3W_donations_per,2W_donations_per,1W_donations_per,mean_donations_per,std_donations_per,1W_normalised,2W_normalised,3W_normalised,4W_normalised,5W_normalised,trend
44776,e31c0ea8b68f404699dfb0d39e9bc99b,2013-12-31,2014-01-07,2014-01-14,2014-01-21,2014-01-28,2014-02-03,6,0.00,0.00,513.41,0.00,0.00,0.000000,0.000000,73.344286,0.000000,0.000000,14.668857,32.800562,-0.447214,-0.447214,1.788854,-0.447214,-0.447214,1.110223e-17
44790,207f67c9d305035930dcc31845801608,2013-12-31,2014-01-07,2014-01-14,2014-01-21,2014-01-28,2014-02-03,6,63.75,114.75,0.00,282.91,0.00,0.000000,40.415714,0.000000,16.392857,9.107143,13.183143,16.704187,-0.244011,0.192150,-0.789212,1.630284,-0.789212,3.477315e-02
44791,d43b5e98aafd4bad872be826176658f3,2013-12-31,2014-01-07,2014-01-14,2014-01-21,2014-01-28,2014-02-03,6,8.50,0.00,0.00,0.00,15.44,2.573333,0.000000,0.000000,0.000000,1.214286,0.757524,1.143167,0.399558,-0.662654,-0.662654,-0.662654,1.588403,2.377689e-01
44804,78aff5ef5f3fb719258f4b4b78e2a796,2013-12-31,2014-01-07,2014-01-14,2014-01-21,2014-01-28,2014-02-03,6,0.00,0.00,42.50,0.00,0.00,0.000000,0.000000,6.071429,0.000000,0.000000,1.214286,2.715225,-0.447214,-0.447214,1.788854,-0.447214,-0.447214,1.110223e-17
44822,22e76148a4a8f4c337d31dd6848732d1,2013-12-31,2014-01-07,2014-01-14,2014-01-21,2014-01-28,2014-02-03,6,0.00,42.50,0.00,0.00,189.48,31.580000,0.000000,0.000000,6.071429,0.000000,7.530286,13.698838,-0.549703,-0.106495,-0.549703,-0.549703,1.755603,4.167403e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663962,67a8a71112f678943e52c1d71fa4c1ad,2002-09-20,2002-09-27,2002-10-04,2002-10-11,2002-10-18,2002-10-21,3,0.00,0.00,0.00,333.00,0.00,0.000000,47.571429,0.000000,0.000000,0.000000,9.514286,21.274590,-0.447214,-0.447214,-0.447214,1.788854,-0.447214,2.236068e-01
663974,6aa0289afe17b68989acf886e634bc51,2002-09-19,2002-09-26,2002-10-03,2002-10-10,2002-10-17,2002-10-21,4,0.00,0.00,0.00,0.00,329.00,82.250000,0.000000,0.000000,0.000000,0.000000,16.450000,36.783318,-0.447214,-0.447214,-0.447214,-0.447214,1.788854,4.472136e-01
664034,e551f861145201ea44608257d32b71b6,2002-09-19,2002-09-26,2002-10-03,2002-10-10,2002-10-17,2002-10-21,4,0.00,0.00,0.00,100.00,0.00,0.000000,14.285714,0.000000,0.000000,0.000000,2.857143,6.388766,-0.447214,-0.447214,-0.447214,1.788854,-0.447214,2.236068e-01
664043,7b49dbcc5697d2dfa18cbb4d603422ff,2002-09-19,2002-09-26,2002-10-03,2002-10-10,2002-10-17,2002-10-21,4,0.00,0.00,234.00,0.00,0.00,0.000000,0.000000,33.428571,0.000000,0.000000,6.685714,14.949712,-0.447214,-0.447214,1.788854,-0.447214,-0.447214,1.110223e-17


In [105]:
trend_df['trend'].max()

0.6317653722655331

In [106]:
trend_df[trend_df['trend'] > 0.5]

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations,4W_donations,5W_donations,5W_donations_per,4W_donations_per,3W_donations_per,2W_donations_per,1W_donations_per,mean_donations_per,std_donations_per,1W_normalised,2W_normalised,3W_normalised,4W_normalised,5W_normalised,trend
45143,6c4eab1b43faf65b20cca6d84e15f607,2013-12-30,2014-01-06,2014-01-13,2014-01-20,2014-01-27,2014-02-03,7,0.00,0.00,17.00,17.00,99.21,14.172857,2.428571,2.428571,0.000000,0.000000,3.806000,5.921098,-0.642786,-0.642786,-0.232631,-0.232631,1.750833,0.519739
45588,f20de845105ac7450ccdf43707295b05,2013-12-27,2014-01-03,2014-01-10,2014-01-17,2014-01-24,2014-01-27,3,0.00,0.00,0.00,123.25,163.75,54.583333,17.607143,0.000000,0.000000,0.000000,14.438095,23.701576,-0.609162,-0.609162,-0.609162,0.133706,1.693779,0.534875
45873,e08ba36aff004e1c17753c860ff2ad82,2013-12-26,2014-01-02,2014-01-09,2014-01-16,2014-01-23,2014-01-27,4,0.00,0.85,0.00,7.10,3.40,0.850000,1.014286,0.000000,0.121429,0.000000,0.397143,0.494320,-0.803413,-0.557765,-0.803413,1.248469,0.916122,0.524530
47216,6713ab4e455d973e1a3e4d7b99f9c44c,2013-12-20,2013-12-27,2014-01-03,2014-01-10,2014-01-17,2014-01-20,3,0.00,0.00,0.00,51.00,110.13,36.710000,7.285714,0.000000,0.000000,0.000000,8.799143,15.918395,-0.552766,-0.552766,-0.552766,-0.095074,1.753371,0.506997
47271,887658fd1f3164b7f043cb386d79e384,2013-12-19,2013-12-26,2014-01-02,2014-01-09,2014-01-16,2014-01-20,4,6.85,2.85,0.85,22.85,26.13,6.532500,3.264286,0.121429,0.407143,0.978571,2.260786,2.688469,-0.476931,-0.689479,-0.795753,0.373261,1.588902,0.519441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661911,96ba6afa02ae5e7ab153d7b1c0e69460,2003-10-10,2003-10-17,2003-10-24,2003-10-31,2003-11-07,2003-11-10,3,0.00,50.00,0.00,92.50,52.50,17.500000,13.214286,0.000000,7.142857,0.000000,7.571429,7.830311,-0.966938,-0.054732,-0.966938,0.720643,1.267966,0.524518
661956,3399722fdf958bfef7fca80341672dba,2003-10-01,2003-10-08,2003-10-15,2003-10-22,2003-10-29,2003-11-03,5,0.00,0.00,0.00,85.00,85.00,17.000000,12.142857,0.000000,0.000000,0.000000,5.828571,8.163758,-0.713957,-0.713957,-0.713957,0.773453,1.368417,0.565216
662118,70ca89336e4162d1e7892632f9971541,2003-09-04,2003-09-11,2003-09-18,2003-09-25,2003-10-02,2003-10-06,4,0.00,0.00,0.00,55.20,95.50,23.875000,7.885714,0.000000,0.000000,0.000000,6.352143,10.373663,-0.612334,-0.612334,-0.612334,0.147833,1.689168,0.536317
662819,1701e6e918803d93f0f47125f7781b94,2003-06-19,2003-06-26,2003-07-03,2003-07-10,2003-07-17,2003-07-21,4,0.00,0.00,0.00,233.75,174.00,43.500000,33.392857,0.000000,0.000000,0.000000,15.378571,21.359018,-0.720004,-0.720004,-0.720004,0.843404,1.316607,0.563663


In [109]:
trend_df[['projectid', 'trend']].to_csv('trend_projedid.csv')

In [110]:
import os

In [113]:
!pwd

/content


In [80]:
# get 2W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp >= projects_donations_filtered_df.date_posted_plus_1_weeks & projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_2_weeks].groupby('projectid').donation_to_project.sum().reset_index(name='2W_donations'), on='projectid', how='left').fillna(0)
# trend_df['2W_donations'] = trend_df['2W_donations'] - trend_df['1W_donations']
trend_df.head()

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0


In [85]:
projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp >= projects_donations_filtered_df.date_posted_plus_1_weeks & projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_2_weeks]

TypeError: ignored

In [81]:
# get 3W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_3_weeks].groupby('projectid').donation_to_project.sum().reset_index(name='3W_donations'), on='projectid', how='left').fillna(0)
trend_df['3W_donations'] = trend_df['3W_donations'] - trend_df['2W_donations']
trend_df.head()

,projectid,date_posted,date_posted_plus_1_weeks,date_posted_plus_2_weeks,date_posted_plus_3_weeks,date_posted_plus_4_weeks,evaluation_date,date_5w_num_days,1W_donations,2W_donations,3W_donations
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0
1,90de744e368a7e4883223ca49318ae30,2014-05-12,2014-05-19,2014-05-26,2014-06-02,2014-06-09,2014-06-16,7,0.0,0.0,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0
4,24761b686e18e5eace634607acbcc19f,2014-05-11,2014-05-18,2014-05-25,2014-06-01,2014-06-08,2014-06-09,1,0.0,0.0,0.0


In [ ]:
# get 4W donations
trend_df = trend_df.merge(projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.date_posted_plus_4_weeks].groupby('projectid').donation_to_project.sum().reset_index(name='4W_donations'), on='projectid', how='left').fillna(0)
trend_df['4W_donations'] = trend_df['4W_donations'] - trend_df['3W_donations']
trend_df.head()

In [57]:
X = -1

def get_trend(r):

  # step 1: get the rows for a pid
  # step 2: get the date posted
  # step 3: break down into 1, 2, 3, 4, 5 weeks (5 only if applicable)
  # step 4: get for each the per day
  # step 5: normalise them
  # step 6: get the trend (return beta)

  pid_df = projects_donations_filtered_df[projects_donations_filtered_df['projectid'] == r.projectid]
  w1_donations = pid_df[pid_df.donation_timestamp < r.date_posted_plus_1_weeks]
  return w1_donations

trend_df.apply(get_trend, axis=1)

KeyboardInterrupt: ignored

In [56]:
(X.date_posted_plus_3_weeks - X.date_posted_plus_4_weeks).days

-7

In [53]:
dir(X.date_posted_plus_4_weeks - X.date_posted_plus_4_weeks)

['__abs__',
 '__add__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__inv__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pos__',
 '__pyx_vtable__',
 '__radd__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_cython__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setstate__',
 '__setstate_cython__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_d',
 '_ensure_components',
 '_h',
 '_has_ns',
 '_m',
 '_ms',
 '_ns',
 '_repr_base',
 '_round',
 '_s',
 '_us',
 'asm8',
 'ceil',
 'components',
 'days',
 'delta',
 'floor',
 'freq',
 'is_populated',
 'isoformat',
 'max',
 'microseconds',
 'min',
 'nanoseconds',
 

In [6]:
# Dataframe of all donations associated with a projectid - filtered by donations within 4-5 weeks of posting
valid_projects_donations_filtered_df = projects_donations_filtered_df[projects_donations_filtered_df.donation_timestamp < projects_donations_filtered_df.evaluation_date]


## Create Donation Level Features

In [7]:
# dataframe with donation features at project-level
donation_features_df = projects[["projectid", "total_price_excluding_optional_support"]]

donation_counts = valid_projects_donations_filtered_df.groupby("projectid").projectid.count().reset_index(name="total_number_of_donations")
donation_values = valid_projects_donations_filtered_df.groupby('projectid')['donation_to_project'].sum().reset_index(name='total_value_of_donations')

In [8]:
num_promo_matched = valid_projects_donations_filtered_df[['projectid', 'payment_was_promo_matched']]
num_promo_matched.payment_was_promo_matched.replace({'f':0, 't':1}, inplace=True)
num_promo_matched = num_promo_matched.groupby("projectid")['payment_was_promo_matched'].sum().reset_index(name='num_promo_matched')

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [9]:
# prop_of_donations_from_same_city
prop_of_donations_from_same_city = valid_projects_donations_filtered_df[['projectid', 'donor_city', 'school_city', 'donation_to_project', 'donor_state', 'school_state']]
prop_of_donations_from_same_city.donor_city = prop_of_donations_from_same_city.donor_city.fillna("NA")
prop_of_donations_from_same_city.donor_state = prop_of_donations_from_same_city.donor_state.fillna("NA")
prop_of_donations_from_same_city['donation_within_city'] = (prop_of_donations_from_same_city.school_city.str.lower() == prop_of_donations_from_same_city.donor_city.str.lower()) & (prop_of_donations_from_same_city.school_state.str.lower() == prop_of_donations_from_same_city.donor_state.str.lower())
prop_of_donations_from_same_city['donation_within_city'] += 0
prop_of_donations_from_same_city = prop_of_donations_from_same_city.groupby(["projectid", "donation_within_city"])['donation_to_project'].sum().reset_index(name='donation_value_within_city')
prop_of_donations_from_same_city = donation_values.merge(
    prop_of_donations_from_same_city[prop_of_donations_from_same_city.donation_within_city == 1], 
    how='left',
    on='projectid'
).fillna(0.0)
prop_of_donations_from_same_city['prop_of_donations_from_same_city'] = prop_of_donations_from_same_city.donation_value_within_city / prop_of_donations_from_same_city.total_value_of_donations
prop_of_donations_from_same_city.prop_of_donations_from_same_city = prop_of_donations_from_same_city.prop_of_donations_from_same_city.fillna(0.0)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-9-c40079fbf798>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_of_donations_from_same_city['donation_within_city'] = (prop_of_donations_from_same_city.school_city.str.lower() == prop_of_donations_from_same_city.donor_city.str.lower()) & (prop_of_donations_from_same_city.school_state.str.lower() == prop_of_donations_from_same_city.donor_state.str.lower())


In [10]:
# prop_of_donations_from_same_state
prop_of_donations_from_same_state = valid_projects_donations_filtered_df[['projectid', 'donor_state', 'school_state', 'donation_to_project']]
prop_of_donations_from_same_state.donor_state = prop_of_donations_from_same_state.donor_state.fillna("NA").replace({'  ': "NA"})
prop_of_donations_from_same_state['donation_within_state'] = prop_of_donations_from_same_state.school_state.str.lower() == prop_of_donations_from_same_state.donor_state.str.lower()
prop_of_donations_from_same_state['donation_within_state'] += 0
prop_of_donations_from_same_state = prop_of_donations_from_same_state.groupby(["projectid", "donation_within_state"])['donation_to_project'].sum().reset_index(name='donation_value_within_state')
prop_of_donations_from_same_state = donation_values.merge(
    prop_of_donations_from_same_state[prop_of_donations_from_same_state.donation_within_state == 1], 
    how='left',
    on='projectid'
).fillna(0.0)
prop_of_donations_from_same_state['prop_of_donations_from_same_state'] = prop_of_donations_from_same_state.donation_value_within_state / prop_of_donations_from_same_state.total_value_of_donations
prop_of_donations_from_same_state.prop_of_donations_from_same_state = prop_of_donations_from_same_state.prop_of_donations_from_same_state.fillna(0.0)

<ipython-input-10-faf8bafec01b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_of_donations_from_same_state['donation_within_state'] = prop_of_donations_from_same_state.school_state.str.lower() == prop_of_donations_from_same_state.donor_state.str.lower()
<ipython-input-10-faf8bafec01b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_of_donations_from_same_state['donation_within_state'] += 0


In [11]:
# prop_of_teacher_donors
prop_of_teacher_donors = valid_projects_donations_filtered_df[['projectid', 'is_teacher_acct', 'donation_to_project']]
prop_of_teacher_donors.is_teacher_acct.replace({'f':0, 't':1}, inplace=True)
prop_of_teacher_donors = prop_of_teacher_donors.groupby(["projectid", "is_teacher_acct"])['donation_to_project'].sum().reset_index(name='donation_value_from_teacher')
prop_of_teacher_donors = donation_values.merge(
    prop_of_teacher_donors[prop_of_teacher_donors.is_teacher_acct == 1], 
    how='left',
    on='projectid'
).fillna(0.0)
prop_of_teacher_donors['prop_of_teacher_donors'] = prop_of_teacher_donors.donation_value_from_teacher / prop_of_teacher_donors.total_value_of_donations
prop_of_teacher_donors.prop_of_teacher_donors = prop_of_teacher_donors.prop_of_teacher_donors.fillna(0.0)

In [12]:
# prop_via_giving_page
prop_via_giving_page = valid_projects_donations_filtered_df[['projectid', 'via_giving_page', 'donation_to_project']]
prop_via_giving_page.via_giving_page.replace({'f':0, 't':1}, inplace=True)
prop_via_giving_page = prop_via_giving_page.groupby(["projectid", "via_giving_page"])['donation_to_project'].sum().reset_index(name='donation_value_via_giving_page')
prop_via_giving_page = donation_values.merge(
    prop_via_giving_page[prop_via_giving_page.via_giving_page == 1], 
    how='left',
    on='projectid'
).fillna(0.0)
prop_via_giving_page['prop_via_giving_page'] = prop_via_giving_page.donation_value_via_giving_page / prop_via_giving_page.total_value_of_donations
prop_via_giving_page.prop_via_giving_page = prop_via_giving_page.prop_via_giving_page.fillna(0.0)

In [15]:
valid_projects_donations_filtered_df

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,donationid,donor_acctid,donor_city,donor_state,donor_zip,is_teacher_acct,donation_timestamp,donation_to_project,donation_optional_support,donation_total,dollar_amount,donation_included_optional_support,payment_method,payment_included_acct_credit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,payment_was_promo_matched,via_giving_page,for_honoree,donation_message,deadline,date_posted_plus_4_weeks,evaluation_date
44774,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,Elgin School District U-46,Du Page,f,f,f,f,f,f,Mrs.,f,f,Special Needs,Special Needs,Literacy,Literacy & Language,Other,moderate poverty,Grades 3-5,30.0,444.36,522.78,7.0,f,f,2013-12-31,cfae0df1b307782f5994dacf28641272,0e22b3e4f6722e5476e41ff6a0708858,NaN,IL,60103.0,f,2014-01-22 13:13:22.777,46.75,8.25,55.0,10_to_100,t,no_cash_received,f,f,t,f,f,f,I live in Bartlett and have 2 kids. I hope thi...,2014-04-30,2014-01-28,2014-02-03
44775,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,Elgin School District U-46,Du Page,f,f,f,f,f,f,Mrs.,f,f,Special Needs,Special Needs,Literacy,Literacy & Language,Other,moderate poverty,Grades 3-5,30.0,444.36,522.78,7.0,f,f,2013-12-31,c0d3d330642ee7eb4ff9fe1af05d4611,4ab8cfa71a54070432a9ee01e9209722,NaN,IL,60118.0,f,2014-01-17 13:39:43.09,21.25,3.75,25.0,10_to_100,t,no_cash_received,t,f,f,f,f,f,All the best!,2014-04-30,2014-01-28,2014-02-03
44777,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,Elgin School District U-46,Du Page,f,f,f,f,f,f,Mrs.,f,f,Special Needs,Special Needs,Literacy,Literacy & Language,Other,moderate poverty,Grades 3-5,30.0,444.36,522.78,7.0,f,f,2013-12-31,c5b1c26c23c32adf923436a280fcecf1,0dd4f7364f9b1b07d4909f33eb2f5d8b,Palatine,IL,60074.0,f,2014-01-05 20:09:37.286,42.50,7.50,50.0,10_to_100,t,no_cash_received,f,f,t,f,f,f,"I've always been a very organized person, and ...",2014-04-30,2014-01-28,2014-02-03
44781,33aa19ee4da4c5adf47d0dfb84fab5ef,17768031eb40de8d4497dbb54df48742,9ac70da58322783f82152eecc140a812,5.103240e+11,37.476158,-77.488397,Richmond,VA,23224.0,urban,Richmond City School District,Richmond City,f,f,f,f,f,f,Ms.,f,f,Literacy,Literacy & Language,NaN,NaN,Other,highest poverty,Grades PreK-2,30.0,232.94,274.05,18.0,f,f,2013-12-31,3f463af5a0630cd4a52e4bd6c4f84b85,6e624cc0ab8b155c07ef2fa99277d3d0,NaN,WV,25812.0,f,2014-01-02 12:38:06.504,42.50,7.50,50.0,10_to_100,t,paypal,f,f,f,f,f,f,NaN,2014-04-30,2014-01-28,2014-02-03
44782,e31c0ea8b68f404699dfb0d39e9bc99b,0f1bc5b4700fd33383be104442660178,cb9f688cf59e3ee22a087d616ca8f5d7,1.709930e+11,41.952851,-87.650233,Chicago,IL,60613.0,urban,Ravenswood-ridge Elem Network,Cook,f,t,f,f,f,f,Mr.,f,f,Environmental Science,Math & Science,NaN,NaN,Supplies,highest poverty,Grades 6-8,30.0,513.41,604.01,70.0,t,f,2013-12-31,649d9ffb4b3a64f687470bba8db914f2,d8701f7bbd3115b477e163a025841c68,San Francisco,CA,94102.0,f,2014-01-20 22:55:41.499,256.70,45.30,302.0,100_and_up,t,double_your_impact_match,f,f,f,f,t,f,Disney is committed to conserving nature and t...,2014-04-30,2014-01-28,2014-0

Merge


In [13]:
# Merge total number of donations
donation_features_df = donation_features_df.merge(donation_counts, on="projectid", how="left").fillna(0)
donation_features_df.total_number_of_donations = donation_features_df.total_number_of_donations.astype(int)

# Merge total value of donations
donation_features_df = donation_features_df.merge(donation_values, on="projectid", how="left").fillna(0)

# Merge num promo matched
donation_features_df = donation_features_df.merge(num_promo_matched, on="projectid", how="left").fillna(0)

# Merge prop_of_donations_from_same_city
donation_features_df = donation_features_df.merge(prop_of_donations_from_same_city[['projectid', 'prop_of_donations_from_same_city']], on="projectid", how="left").fillna(0)

# Merge prop_of_donations_from_same_state
donation_features_df = donation_features_df.merge(prop_of_donations_from_same_state[['projectid', 'prop_of_donations_from_same_state']], on="projectid", how="left").fillna(0)

# Merge prop_of_teacher_donors
donation_features_df = donation_features_df.merge(prop_of_teacher_donors[['projectid', 'prop_of_teacher_donors']], on="projectid", how="left").fillna(0)

# Merge prop_via_giving_page
donation_features_df = donation_features_df.merge(prop_via_giving_page[['projectid', 'prop_via_giving_page']], on="projectid", how="left").fillna(0)

Proportions



In [ ]:
donation_features_df["avg_value_per_donation"] = (donation_features_df["total_value_of_donations"] / donation_features_df["total_number_of_donations"]).fillna(0)

donation_features_df["total_prop_funded"] = (donation_features_df["total_value_of_donations"] / donation_features_df["total_price_excluding_optional_support"]).fillna(1).replace([np.inf, -np.inf], 1.0)

donation_features_df["prop_promo_matched"] = (donation_features_df["num_promo_matched"] / donation_features_df["total_number_of_donations"]).fillna(0)

## Export Donations Features

In [ ]:
donation_features_df.to_csv('Model_Donations_Features_v1.csv')

## Create Project Level Features

In [26]:
project_features_df = projects[['projectid', 'school_state', 'school_metro', 'school_charter', 'school_magnet', 'school_year_round', 'school_nlns',
       'school_kipp', 'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow', 'primary_focus_area', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'total_price_excluding_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match']]

## Imputing project level features

In [ ]:
missing_str = "missing"

# Replace La school state code with LA
project_features_df.school_state.replace({'La': 'LA'}, inplace=True)

# Replace school metro nan with missing
project_features_df.school_metro.fillna(missing_str, inplace=True)

# Replace teacher prefix nan with missing
project_features_df.teacher_prefix.fillna(missing_str, inplace=True)

# Replace primary focus area nan with missing
project_features_df.primary_focus_area.fillna(missing_str, inplace=True)

# Replace secondary focus area nan with missing
project_features_df.secondary_focus_area.fillna(missing_str, inplace=True)

# Replace resource type nan with missing
project_features_df.resource_type.fillna(missing_str, inplace=True)

# Replace grade level nan with missing
project_features_df.grade_level.fillna(missing_str, inplace=True)

# Replace students reached with 0
project_features_df.students_reached.fillna(0, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


## Convert to dummy variables

In [ ]:
dummy_columns = ['school_state', 'school_metro', 'teacher_prefix', 'primary_focus_area', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level']
project_features_dummy_df = pd.get_dummies(project_features_df, columns=dummy_columns)

# Replace 'f' with 0 and 't' with 1
project_features_dummy_df.replace({'f': 0, 't': 1}, inplace=True)

In [ ]:
# add basic columns to project df
# projects['date_posted'] = projects.date_posted.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# projects['date_posted_plus_4_weeks'] = projects['date_posted'] + pd.DateOffset(weeks=4)
# projects['evaluation_date'] = projects.date_posted_plus_4_weeks.apply(lambda x: next_weekday(x, 0))

## Export Projects Features

In [ ]:
project_features_dummy_df.to_csv('Model_Projects_Features_v1.csv')

## Create Essay Level Features

In [ ]:
essay_features_df = essays[['projectid', 'title', 'short_description', 'essay']]

# # num_words_in_title
essay_features_df.title = essay_features_df.title.fillna("")
essay_features_df['num_words_in_title'] = essay_features_df.title.str.split().apply(len)

# # num_words_in_short_description
essay_features_df.short_description = essay_features_df.short_description.fillna("")
essay_features_df['num_words_in_short_description'] = essay_features_df.short_description.str.split().apply(len)

<ipython-input-6-e3152b0f72f6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_features_df['num_words_in_title'] = essay_features_df.title.str.split().apply(len)


In [ ]:
# num_words_in_essay
essay_features_df.essay = essay_features_df.essay.fillna("")
# essay_features_df['num_words_in_essay'] = essay_features_df.essay.str.split().apply(len)

# essay_features_df = essay_features_df[['projectid', 'num_words_in_title', 'num_words_in_short_description', 'num_words_in_essay']]

In [ ]:
essay_features_df['num_words_in_essay'] = 0
for i in tqdm(range(len(essay_features_df)//1000 + 1)):
  essay_features_df.iloc[i * 1000 : (i+1) * 1000]['num_words_in_essay'] = essay_features_df.iloc[i * 1000 : (i+1) * 1000]['essay'].str.split().apply(len)

  0%|          | 0/665 [00:00<?, ?it/s]<ipython-input-23-0cfba8dcf4ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_features_df.iloc[i * 1000 : (i+1) * 1000]['num_words_in_essay'] = essay_features_df.iloc[i * 1000 : (i+1) * 1000]['essay'].str.split().apply(len)
100%|██████████| 665/665 [00:20<00:00, 31.85it/s]


In [ ]:
essay_features_df = essay_features_df[['projectid', 'num_words_in_title', 'num_words_in_short_description', 'num_words_in_essay']]

## Export Essay Features

In [ ]:
essay_features_df.to_csv('Model_Essay_Features_v1.csv')

## Merge Features

In [ ]:
project_features_dummy_df = pd.read_csv('./gdrive/MyDrive/SPARSe/features_labels/Model_Projects_Features_v1.csv').drop(['Unnamed: 0'], axis=1)

In [ ]:
donation_features_df = pd.read_csv('./gdrive/MyDrive/SPARSe/features_labels/Model_Donations_Features_v1.csv').drop(['Unnamed: 0'], axis=1)

In [ ]:
essay_features_df = pd.read_csv('./gdrive/MyDrive/SPARSe/features_labels/Model_Essay_Features_v1.csv').drop(['Unnamed: 0'], axis=1)

In [ ]:
features_df = project_features_dummy_df.merge(donation_features_df, on=['projectid', 'total_price_excluding_optional_support'], how='inner')

In [ ]:
features_df = features_df.merge(essay_features_df, on='projectid', how='left')

In [ ]:
features_df.to_csv('Model_Features_v1.csv')

## Create Labels

In [ ]:
funded_amt_df = projects_donations_filtered_df.groupby('projectid')['donation_to_project'].sum().reset_index(name='funded_amt')
funding_frac_df = projects.merge(funded_amt_df, on='projectid', how='left')[['projectid', 'funded_amt', 'total_price_excluding_optional_support']].fillna(0.0)
funding_frac_df['funding_frac'] = funding_frac_df.funded_amt / funding_frac_df.total_price_excluding_optional_support
funding_frac_df = funding_frac_df.fillna(1.0)
funding_frac_df.replace([np.inf, -np.inf], 1.0, inplace=True)
funding_frac_df['is_fully_funded'] = funding_frac_df.funding_frac >= 1.0

In [ ]:
labels_binary_df = funding_frac_df[['projectid', 'is_fully_funded']].replace({False: 0, True: 1}, inplace=False)
labels_binary_df.to_csv('Model_Labels_Binary_v0.csv')

In [ ]:
labels_continuous_df = funding_frac_df[['projectid', 'funding_frac']]
labels_continuous_df.to_csv('Model_Labels_Continuous_v0.csv')

## Create Train/Test Splits

In [ ]:
train_test_val_project_ids = json.load(open('update2_baseline_3trainvalsplit-enddate_2013_11_24-10wk.json', 'r'))

In [ ]:
features_val3df = features_df[features_df.projectid.isin(train_test_val_project_ids['valid3'])]
features_train3df = features_df[features_df.projectid.isin(train_test_val_project_ids['train3'])]

labels_val3df = labels_df[labels_df.projectid.isin(train_test_val_project_ids['valid3'])]
labels_train3df = labels_df[labels_df.projectid.isin(train_test_val_project_ids['train3'])]

In [ ]:
features_val3df.to_csv('Features_val3.csv')
features_train3df.to_csv('Features_train3.csv')
labels_val3df.to_csv('Labels_val3.csv')
labels_train3df.to_csv('Labels_train3.csv')